<a href="https://colab.research.google.com/github/seopbo/nlp_tutorials/blob/main/question_answering_(klue_mrc)_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question answering - BERT
- pre-trained language model로는 `klue/bert-base`를 사용합니다.
  - https://huggingface.co/klue/bert-base
- extractive question asnwering을 수행하는 예시 데이터셋으로는 klue의 mrc를 사용합니다.


## Setup
어떠한 GPU가 할당되었는 지 아래의 코드 셀을 실행함으로써 확인할 수 있습니다.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Tue Dec 28 01:32:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    38W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

아래의 코드 셀을 실행함으로써 본 노트북을 실행하기위한 library를 install하고 load합니다.

In [2]:
!pip install torch
!pip install transformers
!pip install datasets

from pprint import pprint
import torch
import transformers
import datasets

## Preprocess data
1. `klue/bert-base`가 사용한 subword tokenizer를 load합니다.
2. `datasets` library를 이용하여 klue의 mrc를 load합니다.
3. 1의 subword tokenizer를 이용 klue mrc의 data를 span detection을 수행할 수 있는 형태, train example로 transformation합니다.

- `[CLS] question_tokens [SEP] context_tokens [SEP]`로 만들고, 이를 list_of_integers로 transform합니다.
- 단 주어진 model의 `max_sequence_length` list_of_integers를 처리할 수 있는 길이가 아닐 경우에 아래의 조치를 수행합니다.
  - `question_tokens`는 상대적으로 짧고 `context_tokens`는 상대적으로 길이가 길 것이기 때문에 truncation이 발생할 경우, `context_tokens`만 truncation이 이루어지도록 합니다 tokenizer의 `__call__` method 활용 시, `truncation` parameter에 `"only_second"`를 전달하여 위와 같은 동작을 수행할 수 있습니다.
  - 위의 동작을 수행 시 question에 대한 답이 잘려 나갈 가능성이 있습니다. tokenizer의 `__call__` method 활용 시, `stride` parameter에 적절한 값을 주어 `context_tokens`를 겹쳐보게 만들고, `return_overflowing_tokens`에 `True`를 전달하여, 답이 잘려 나갈 가능성을 방지할 수 있습니다. 단 이 경우 하나의 (question, context)에서 여러개의 training example이 생성될 수 있습니다.

- answer가 context에서 어디에 위치하는 지 확인합니다. 특히 우리가 사용하는 tokenizer의 결과에 맞추어 tokenizer의 결과에서 몇 개의 연속된 token이 answer와 일치하는 지에 대한 정보를 만들어주어야합니다. tokenizer의 `__call__` method 활용 시, `return_offsets_mapping` parameter에 `True`를 전달하여 얻은 offset 정보로 위의 정보를 생성해야합니다.


In [3]:
from transformers import AutoTokenizer, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

print(tokenizer.__class__)

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [4]:
from datasets import load_dataset

cs = load_dataset("klue", "mrc", split="train")
# 대답할 수 없는 경우는 제외
cs = cs.filter(lambda example: example["is_impossible"] == False)
cs = cs.train_test_split(0.1)

train_cs = cs["train"]
valid_cs = cs["test"]

test_cs = load_dataset("klue", "mrc", split="validation")
test_cs = test_cs.filter(lambda example: example["is_impossible"] == False)

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)
Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-8775884b4f525c7f.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-a30f2529f0b3c2c2.arrow and /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-58bb360da3c80e7c.arrow
Reusing dataset klue (/root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)
Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-4cfb9e85dc37f419.arrow


위에서 말한 내용을 실제로 동작시켜보며 알아보면 아래와 같습니다.

In [5]:
example = train_cs[0]
pprint(example)

{'answers': {'answer_start': [12], 'text': ['류재선']},
 'context': '한국전기공사협회(회장 류재선)와 전기공사공제조합(이사장 김성관)은 에너지시민연대와 공동으로 7월 16일 서울 명동에서 '
            '‘에너지 절약 캠페인’을 실시했다. 이번 캠페인은 명동 일대 상점과 시민들에게 안전하고 효율적인 전기사용의 중요성을 '
            '알리고 에너지 절약의 필요성을 전파하기 위한 것으로, 전력산업계를 대표하는 협회, 조합, 전기신문사, 전기산업연구원, '
            '전기공사공제조합장학회, 엘비라이프 등 각 기관 임직원과 대학생 자원봉사자 200여명이 함께해 행사의 의미를 더했다. '
            '참가자들은 행사 시작에 앞서 전기절약의 중요성을 담은 퍼포먼스를 펼친 후, 명동 일대를 행진하며 시민들이 직접 실천할 '
            '수 있는 내용이 담긴 부채와 쿨 스카프를 전파했다. 또 각 상점을 방문해 안전하고 효율적인 전기기기 사용법을 전파하며 '
            '에너지 절약 실천을 당부했다. 이날 전기공사협회 류재선 회장은 “올바른 전기 사용의 중요성이 많이 홍보됐지만 아직 '
            '위험하고 비효율적으로 사용되고 있다”며 시민들의 관심과 전기 절약 노력을 당부했다. 이어 김성관 전기공사공제조합 '
            '이사장은 “안정적인 전력 공급을 위해 전력사용량이 급증하는 여름철 효율적인 냉방 등으로 에너지절약을 실천해 주시길 '
            '바란다”고 말했다. 한국전기공사협회는 이번 캠페인이 안정적인 전력공급을 위한 에너지절약에 전기건설산업계가 앞장섰다는 '
            '점에서 의미가 있다고 밝혔다.',
 'guid': 'klue-mrc-v1_train_15747',
 'is_impossible': False,
 'news_category': '산업',
 'question': '캠페인의 의의를 언급한 단체의

In [6]:
# truncation parameter에 따른 차이
print(tokenizer.convert_ids_to_tokens(tokenizer(example["question"], example["context"], truncation=True, max_length=24)["input_ids"]))
print(tokenizer.convert_ids_to_tokens(tokenizer(example["question"], example["context"], truncation="only_second", max_length=24)["input_ids"]))

['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '[SEP]', '한국', '##전기', '##공사', '##협회', '(', '회장', '류', '##재', '##선', ')', '와', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '한국', '##전기', '##공사', '##협회', '(', '회장', '류', '##재', '##선', ')', '[SEP]']


In [7]:
# return_overflowing_tokens=True의 결과
tokenized_example = tokenizer(example["question"], example["context"], truncation="only_second", max_length=24, return_overflowing_tokens=True)

for input_ids in tokenized_example["input_ids"]:
    print(tokenizer.convert_ids_to_tokens(input_ids))

['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '한국', '##전기', '##공사', '##협회', '(', '회장', '류', '##재', '##선', ')', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '와', '전기', '##공사', '##공', '##제', '##조합', '(', '이사장', '김성', '##관', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', ')', '은', '에너지', '##시', '##민', '##연대', '##와', '공동', '##으로', '7', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '##월', '16', '##일', '서울', '명동', '##에서', '‘', '에너지', '절약', '캠페인', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '’', '을', '실시', '##했', '##다', '.', '이번', '캠페인', '##은', '명동', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '일대', '상점', '##과', '시민', '##들', '##에', '##게', '안전', '##하고', '효율', '[SEP]']
['[CLS]', '캠페인', '

In [8]:
# stride에 적절한 값을 전달 시
tokenized_example = tokenizer(example["question"], example["context"], truncation="only_second", stride=4, max_length=24, return_overflowing_tokens=True)

for input_ids in tokenized_example["input_ids"]:
    print(tokenizer.convert_ids_to_tokens(input_ids))

['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '한국', '##전기', '##공사', '##협회', '(', '회장', '류', '##재', '##선', ')', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '류', '##재', '##선', ')', '와', '전기', '##공사', '##공', '##제', '##조합', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '##공사', '##공', '##제', '##조합', '(', '이사장', '김성', '##관', ')', '은', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '김성', '##관', ')', '은', '에너지', '##시', '##민', '##연대', '##와', '공동', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '##민', '##연대', '##와', '공동', '##으로', '7', '##월', '16', '##일', '서울', '[SEP]']
['[CLS]', '캠페인', '##의', '의의', '##를', '언급', '##한', '단체', '##의', '대표', '##는', '?', '[SEP]', '##월', '16', '##일', '서울', '명동', '##에서', '‘', '에너지', '절약', '캠페인', '[SEP]']
['[CLS]', '캠페인

동작을 충분히 확인하였으므로, answer가 tokenized context를 구성하는 token들 중 어떤 token들에 해당하는 지 확인하는 코드를 작성해보겠습니다.

In [9]:
# 먼저 우리가 사용할 klue/bert-base가 처리할 수 있는 max_sequence_length를 확인합니다.
from transformers import AutoConfig

config = AutoConfig.from_pretrained("klue/bert-base")
print(config.max_position_embeddings)

512


In [10]:
# klue/bert-base가 처리할 수 있는 max_sequence_length보다 긴 길이를 지닌 example을 확인합니다.
for i, example in enumerate(train_cs):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 512:
        break
example = train_cs[i]
# example = train_cs[3]
pprint(example)

Token indices sequence length is longer than the specified maximum sequence length for this model (809 > 512). Running this sequence through the model will result in indexing errors


{'answers': {'answer_start': [198], 'text': ['사비나미술관']},
 'context': '미래학자 앨빈 토플러는 2006년 저서 《부의 미래》에서 “3D프린터는 상상하지 못했던 그 무엇이든 만들어낼 수 '
            '있다”고 예측했다. 불과 8년이 지난 지금, 그의 말대로 3D(3차원)프린터만 있으면 누구든 자신이 원하는 물건을 '
            '디자인해 복잡한 공정을 거치지 않고 제작할 수 있게 됐다. 예술 창작 패러다임에도 커다란 변화의 바람이 불고 '
            '있다.서울 안국동 사비나미술관은 이 ‘21세기의 연금술’이 미술에 미치게 될 영향과 미래를 가늠하기 위한 기획전을 '
            '연다. 15일부터 7월6일까지 열리는 ‘3D프린팅과 예술: 예술가의 새로운 창작도구’전이다. 예술에서의 3D프린터 '
            '활용 가능성은 이미 2000년대 초 일부 학자들에 의해 제기됐다. 그러나 일반에 알려지기 시작한 것은 2007년 '
            '월스트리트저널, 타임매거진 등 세계적 언론이 앞다퉈 보도하면서부터다. 19세기 초 사진이 등장해 예술의 패러다임을 '
            '획기적으로 바꿔놨듯이 3D프린터는 형태 제작의 한계와 장르의 경계를 무너뜨리고 예술 창조의 영역을 무한대로 확장시킬 '
            '것이라는 전망을 쏟아냈다. 특히 상상 속에서만 가능하고 실제로는 구현이 불가능한 형태도 만들어 낼 수 있다는 점은 '
            '예술가들에게 희망을 심어줬다. 국내에선 3D프린터의 개발과 보급이 상대적으로 더뎌 예술가들에게는 아직 낯선 게 현실. '
            '사비나미술관은 이런 상황을 감안해 대림화학으로부터 보급용 3D프린터와 필라멘트(재료)를 무상으로 지원받는 한편 여러 '
            '차례 워크숍을 열어 작가와 토론을 거치며 전시를 꾸렸다.이번 전시에는 3D프린터에 관심을 갖고 있는 신진 및 중

In [11]:
# 위의 parameter 동작에서 확인한 내용을 적용하여 tokenizer의 __call__ method를 활용합니다.
stride=128
tokenized_example = tokenizer(example["question"], example["context"], truncation="only_second", return_offsets_mapping=True, return_overflowing_tokens=True, stride=stride)

In [12]:
# 보통 answer text가 context에서 시작하는 위치정보를 주기 때문에, 아래의 코드로 context에서 어디서부터 어디까지가 answer text에 해당하는 지 확인할 수 있습니다.
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])
print(start_char, end_char)
print(example["context"][start_char:end_char])

198 204
사비나미술관


In [13]:
# tokenized_example의 sequence_ids method를 활용합니다. 이를 통해서 특정 token이 question에서 왔는 지 context에서 왔는 지 확인할 수 있습니다.
# 결과에서 None은 special_token(e.g. [CLS])을 가리킵니다.
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [14]:
# tokenized_example의 input_ids 결과에서 context의 시작과 끝 index를 확인할 수 있습니다.
# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1
# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1
print(token_start_index, token_end_index)

26 510


In [15]:
# offset mapping을 이용하여 answer가 context에 존재하면 token level로 위치를 mapping하고, answer가 context에 존재하지않으면 가장 맨 앞의 [CLS]를 가리키게합니다. 
# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]

if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    # 아래는 제가 생각한 것
    # while token_start_index < len(offsets) and offsets[token_start_index][0] < start_char:
    #     token_start_index += 1
    # start_position = token_start_index
    # while offsets[token_end_index][1] > end_char:
    #     token_end_index -= 1
    # end_position = token_end_index
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

if start_position == end_position:
  print(tokenizer.convert_ids_to_tokens(tokenized_example["input_ids"][0])[start_position:end_position+1])
else:
  print(tokenizer.convert_ids_to_tokens(tokenized_example["input_ids"][0])[start_position:end_position])

134 137
['사비', '##나', '##미술']


이 결과를 일반하여 다음의 함수를 작성합니다.

In [16]:
from typing import Union, List


def prepare_train_features(examples, tokenizer, max_length=512, stride=128):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [17]:
train_ds = train_cs.map(
    lambda examples: prepare_train_features(examples, tokenizer, max_length=512, stride=128),
    remove_columns=train_cs.column_names,
    batched=True
)
valid_ds = valid_cs.map(
    lambda examples: prepare_train_features(examples, tokenizer, max_length=512, stride=128),
    remove_columns=valid_cs.column_names,
    batched=True
)
test_ds = test_cs.map(
    lambda examples: prepare_train_features(examples, tokenizer, max_length=512, stride=128),
    remove_columns=test_cs.column_names,
    batched=True
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-feb750ed450ae791.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-1bd595d2335dde17.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-1e1b90a8962ce49f.arrow


## Prepare model
extractive question answering을 수행하기위해서 `klue/bert-base` load합니다.

In [18]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("klue/bert-base")

print(model.__class__)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

<class 'transformers.models.bert.modeling_bert.BertForQuestionAnswering'>


## Train model
`Trainer` class를 이용하여 train합니다.

- https://huggingface.co/transformers/custom_datasets.html?highlight=trainer#fine-tuning-with-trainer

In [19]:
from transformers.data.data_collator import DataCollatorWithPadding
from datasets import load_metric

batchify = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True
)

In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',     
    evaluation_strategy="epoch",
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    weight_decay=0.01,
    adam_beta1=.9,
    adam_beta2=.95,
    adam_epsilon=1e-8,
    max_grad_norm=1.,
    num_train_epochs=2,
    lr_scheduler_type="linear",
    warmup_steps=100,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=100,
    save_strategy="epoch",
    seed=42,
    dataloader_drop_last=False,
    dataloader_num_workers=2
)

trainer = Trainer(
    args=training_args,
    data_collator=batchify,
    model=model,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
)

trainer.train()

***** Running training *****
  Num examples = 16894
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2112


Epoch,Training Loss,Validation Loss
1,1.060200,1.038632
2,0.391900,1.054650


***** Running Evaluation *****
  Num examples = 1845
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1056
Configuration saved in ./results/checkpoint-1056/config.json
Model weights saved in ./results/checkpoint-1056/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1845
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-2112
Configuration saved in ./results/checkpoint-2112/config.json
Model weights saved in ./results/checkpoint-2112/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2112, training_loss=0.9721131189302965, metrics={'train_runtime': 1908.2646, 'train_samples_per_second': 17.706, 'train_steps_per_second': 1.107, 'total_flos': 8828368195650048.0, 'train_loss': 0.9721131189302965, 'epoch': 2.0})

## Evaulate model
extractive question answering을 수행하는 model을 평가하기위해서는 다소 복잡한 아래의 과정이 필요합니다.

- model이 token 별로 예측한 start, end를 `context`의 token에 mapping하는 과정

이 과정을 어떻게 함수화해야하는 지를 예제를 통해서 알아보도록 합니다.

In [21]:
# mini-batch를 하나 꺼내와서 테스트
for batch in trainer.get_test_dataloader(test_ds):
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

output을 확인해보면 sequence_of_tokens의 token 별로 `start_logits`과 `end_logits`을 가지고 있음을 알 수 있습니다. 

- https://huggingface.co/transformers/_modules/transformers/models/bert/modeling_bert.html#BertForQuestionAnswering

In [22]:
output.start_logits.shape, output.end_logits.shape # [16,512,2] -> [16,512,1], [16,512,1] =[16,512], [16,512]

(torch.Size([16, 512]), torch.Size([16, 512]))

각각의 token에 `start_position`인지 `end_position`인지 예측한 score인 `start_logits`, `end_logits`에 대해서, `start_logits`, `end_logits`에 각각 argmax를 하여 mini-batch를 구성하는 각각의 training example의 `start_position`과 `end_position`을 구합니다.

In [23]:
print(output.start_logits.argmax(dim=-1), output.start_logits.argmax(dim=-1).shape)
print(output.end_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1).shape)

tensor([352, 301, 296,   0, 297,   0, 217, 439,  67, 397,   0, 124,   0, 325,
        197,   0], device='cuda:0') torch.Size([16])
tensor([359, 305, 300,   0, 299,   0, 217, 442,  70, 399,   0, 128,   0, 325,
        197,   0], device='cuda:0') torch.Size([16])


하지만 위의 경우는 아래의 경우에 대해서 대처하지 못할 경우가 있습니다.

- `start_position`이 `end_position`보다 뒤에 예측되는 경우
- `start_position`과 `end_position`이 question 쪽이 찍히는 경우 

따라서 아래와 같이 처리할 필요성이 있습니다.

> To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [24]:
import numpy as np
n_best_size = 20
# mini-batch 중 하나의 training_example을 가지고와서 예시코드 작성
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )
pprint(valid_answers[:5])

[{'score': 12.636349, 'text': ''},
 {'score': 10.846488, 'text': ''},
 {'score': 9.155521, 'text': ''},
 {'score': 5.8207817, 'text': ''},
 {'score': 5.6747694, 'text': ''}]


위의 결과에서는 `score`에 맞는 text span을 아직 가져오지 않았습니다. 이에 대응할 수 있도록 아래의 사항을 고려한 작업을 수행해야합니다.

- `question`이 아닌 `context`에서 valid span을 가져온다.

> To do this, we need to add two things to our validation features:
 - the ID of the example that generated the feature (since each example can generate several features, as seen before);
 - the offset mapping that will give us a map from token indices to character positions in the context.

이 과정을 처리하는 과정을 예시로 작성해봅니다.

In [25]:
# Some of the questions have lots of whitespace on the left, which is not useful and will make the
# truncation of the context fail (the tokenized question will take a lots of space). So we remove that
# left whitespace

test_example = test_cs[:1]
test_example["question"] = [q.lstrip() for q in test_example["question"]]
pprint(test_example)

{'answers': [{'answer_start': [666, 666],
              'text': ['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']}],
 'context': ['BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. '
             '이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 '
             '4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 '
             '신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 '
             '옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 '
             '그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 '
             '것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 '
             '있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 '
             '컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 '
             '자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 '
             '25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 '
             '그린 다크 색상

In [26]:
# Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
# in one example possible giving several features when a context is long, each of those features having a
# context that overlaps a bit the context of the previous feature.
tokenized_test_example = tokenizer(
    test_example["question"],
    test_example["context"],
    truncation="only_second",
    max_length=512,
    stride=128,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

In [27]:
# Since one example might give us several features if it has a long context, we need a map from a feature to
# its corresponding example. This key gives us just that.
sample_mapping = tokenized_test_example.pop("overflow_to_sample_mapping")

In [28]:
# We keep the example_id that gave us this feature and we will store the offset mappings.
tokenized_test_example["example_id"] = []

for i in range(len(tokenized_test_example["input_ids"])):
    # Grab the sequence corresponding to that example (to know what is the context and what is the question).
    sequence_ids = tokenized_test_example.sequence_ids(i)
    context_index = 1
    # One example can give several spans, this is the index of the example containing this span of text.
    sample_index = sample_mapping[i]
    tokenized_test_example["example_id"].append(test_example["guid"][sample_index])

    # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
    # position is part of the context or not.
    tokenized_test_example["offset_mapping"][i] = [
        (o if sequence_ids[k] == context_index else None)
        for k, o in enumerate(tokenized_test_example["offset_mapping"][i])
    ]

print(tokenized_test_example["example_id"])
print(tokenized_test_example["offset_mapping"])

['klue-mrc-v1_dev_01891']
[[None, None, None, None, None, None, None, None, None, None, None, None, None, None, (0, 3), (4, 7), (7, 8), (8, 10), (11, 13), (13, 14), (14, 15), (15, 16), (17, 19), (20, 22), (22, 23), (23, 24), (24, 25), (26, 28), (28, 29), (29, 30), (31, 32), (32, 35), (36, 39), (40, 42), (42, 43), (43, 44), (45, 48), (48, 49), (49, 50), (51, 53), (54, 56), (56, 58), (58, 59), (60, 62), (62, 63), (63, 64), (65, 67), (68, 71), (72, 75), (76, 78), (78, 79), (79, 80), (81, 84), (84, 85), (85, 87), (88, 90), (90, 91), (91, 92), (93, 96), (96, 97), (97, 98), (99, 102), (103, 104), (104, 105), (105, 106), (106, 107), (107, 108), (109, 110), (110, 111), (111, 112), (112, 113), (113, 114), (115, 116), (116, 117), (117, 118), (118, 119), (119, 120), (121, 122), (122, 123), (123, 124), (124, 125), (126, 127), (128, 129), (129, 130), (130, 131), (132, 133), (133, 134), (135, 137), (137, 138), (139, 141), (141, 142), (142, 143), (143, 144), (145, 148), (149, 152), (153, 155), (155, 

In [29]:
def prepare_validation_features(examples, tokenizer, max_length=512, stride=128):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["guid"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [30]:
test_features = test_cs.map(
    lambda examples: prepare_validation_features(examples, tokenizer, max_length=512, stride=128),
    batched=True,
    remove_columns=test_cs.column_names
)

  0%|          | 0/5 [00:00<?, ?ba/s]

새롭게 정의한 `prepare_validation_features` function을 이용해서 transform된 example에 대해서 prediction을 수행합니다.

In [31]:
test_predictions = trainer.predict(test_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 6268
  Batch size = 16


~*The* `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:~

In [ ]:
# test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [32]:
max_answer_length = 30

In [33]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = test_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = test_cs[0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 12.636349, 'text': '뉴 740Li 25주년 에디션'},
 {'score': 10.846488, 'text': '뉴 740Li 25주년 에디션(7대 한정)'},
 {'score': 9.155521, 'text': '뉴 740Li'},
 {'score': 8.717138, 'text': '25주년 에디션'},
 {'score': 6.9272776, 'text': '25주년 에디션(7대 한정)'},
 {'score': 6.6313076, 'text': '(7대 한정)'},
 {'score': 5.6747694, 'text': '뉴 740Li 25주년'},
 {'score': 4.7079053, 'text': '뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크'},
 {'score': 4.456806, 'text': '740Li 25주년 에디션'},
 {'score': 4.151575, 'text': '뉴 740Li 25'},
 {'score': 3.9307623,
  'text': '뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션'},
 {'score': 2.666946, 'text': '740Li 25주년 에디션(7대 한정)'},
 {'score': 1.7555588, 'text': '25주년'},
 {'score': 0.97597945, 'text': '740Li'},
 {'score': 0.953266, 'text': '뉴 530i 25주년 에디션'},
 {'score': 0.788695, 'text': '25주년 에디션(7대 한정)에는 말라카이트 그린 다크'},
 {'score': 0.492725, 'text': '(7대 한정)에는 말라카이트 그린 다크'},
 {'score': 0.44993544,
  'text': '뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li'},
 {'score': 0.2323649, 'text

We can compare to the actual ground-truth answer:

In [34]:
test_cs[0]["answers"]

{'answer_start': [666, 666], 'text': ['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']}

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [35]:
import collections

examples = test_cs
features = test_features

example_id_to_index = {k: i for i, k in enumerate(examples["guid"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [36]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["guid"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
        #     predictions[example["id"]] = best_answer["text"]
        # else:
        #     answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        #     predictions[example["id"]] = answer
        predictions[example["guid"]] = best_answer["text"]

    return predictions

And we can apply our post-processing function to our raw predictions:

In [37]:
final_predictions = postprocess_qa_predictions(test_cs, test_features, test_predictions.predictions)

Post-processing 4008 example predictions split into 6268 features.


  0%|          | 0/4008 [00:00<?, ?it/s]

Then we can load the metric from the datasets library.

In [38]:
# metric = load_metric("squad_v2" if squad_v2 else "squad")
metric = load_metric("squad")

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [39]:
# if squad_v2:
#     formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
# else:
#     formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["guid"], "answers": ex["answers"]} for ex in test_cs]
metric.compute(predictions=formatted_predictions, references=references, )

{'exact_match': 69.76047904191617, 'f1': 73.7957510336649}